In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

In [ ]:
# System Parameters (in units of )
#-----------------------------------
Nc = 4                      # Number of cavity states
Nm = 80                      # Number of mech states

kappa = 0.3                 # Cavity damping rate

E = 0.1                     # Driving Amplitude         
g0 = 2.4*kappa              # Coupling strength
Qm = 1e4                    # Mech quality factor
gamma = 1/Qm                # Mech damping rate
n_th = 1                    # Mech bath temperature

hbar = 1                    # Planck constant - bar

wc = 1.0;                   # Cavity Resonant Frequency
w0 = wc/100;                # Mechanical Resonant Frequency (As a fraction of wc)

In [ ]:
# Operators
#----------
c = tensor(destroy(Nc), qeye(Nm)) # annihilation for cavity
a = tensor(qeye(Nc), destroy(Nm)) # annihilation for oscillator
num_a = a.dag()*a                 # number op. for oscillator
num_c = c.dag()*c                 # number op. for cavity

# Initial state density matrix
#-------

thermC = thermal_dm(Nc,Nc/2)   # thermal denisty matrix with (first_arg) states with avg. occupation given by (second arg)
thermM = thermal_dm(Nm,Nm/2)

therm0 = tensor(thermC,thermM)



In [ ]:
wps = [wc-2*w0, wc-w0, wc, wc+w0, wc+2*w0]; # Pump Frequency (As a fraction of w0)

for i in range(len(wps)):  # Loop through all values for cavity frequency we specified in above list       
    delta = wps[i]-wc;               # Detuning

    # Hamiltonian
    #------------
    H = hbar*((delta)*(num_c) + num_a+ g0*(a.dag()+a)*num_c + 1j*E*(c.dag()-c))
    #------------

    times = np.linspace(0.0, 100.0, 101)
    
    expec_ops = [num_a,num_c] # Operators you want to find the expectation values for

    result = mesolve(H, therm0, times, [], expec_ops)
    
    expecs = result.expect # extract expectation values for num_a,num_c over time
    
    if i == 0:
        A = expecs;
    else:
        A = np.vstack((A,expecs));
    

In [ ]:
plt.hold(True)

colours = ['b','g','r','c','m','y','k']

for i in range(len(wps)): # loop over number of cavity frequencies we did simulations for
    
    colourstr_m = colours[i] + '.'
    colourstr_c = colours[i] + 'x'
    
    plt.plot(result.times, A[2*(i),:],colourstr_m);

    plt.plot(result.times, A[2*(i)+1,:],colourstr_c);

plt.xlabel('Time');
plt.ylabel('Expectation values');

plt.show()